In [47]:
import pandas as pd
import glob
import torchaudio
import re
SAMPLE_RATE=16000

In [14]:
audios = glob.glob('../../data/train/*.wav')

In [10]:
df = pd.DataFrame()

In [58]:
str_exemple = audios[0]
print(str_exemple)
pattern = ".*train/(?P<sess_id>.*)_(?P<utt_id>.*)_(?P<spk_id>.*).wav"
print('pattern:',pattern)

../../data/train/bfamcv01_segment163_neutral.wav
pattern: .*train/(?P<sess_id>.*)_(?P<utt_id>.*)_(?P<spk_id>.*).wav


In [101]:
df = pd.DataFrame({'audio' : audios})
df = pd.concat([df,df.audio.str.extract(pattern)], axis = 1)
df.head(1)

,audio,sess_id,utt_id,spk_id
0,../../data/train/bfamcv01_segment163_neutral.wav,bfamcv01,segment163,neutral


In [103]:
df['ID'] = df.apply(lambda row: "--".join([row.spk_id, row.sess_id, row.utt_id]) , axis = 1)
df['start'] = 0
df['stop'] = df.audio.apply(lambda x: torchaudio.load(x)[0].shape[1])
df['duration'] = df.stop.apply(lambda x: x/SAMPLE_RATE).astype(str)
df['wav'] = df.audio

In [104]:
df = df[["ID", "duration", "wav", "start", "stop", "spk_id"]]

In [105]:
df

,ID,duration,wav,start,stop,spk_id
0,neutral--bfamcv01--segment163,5.791,../../data/train/bfamcv01_segment163_neutral.wav,0,92656,neutral
1,non-neutral-male--bfamcv01--segment168,4.704,../../data/train/bfamcv01_segment168_non-neutr...,0,75264,non-neutral-male
2,non-neutral-male--bfamcv01--segment170,2.027,../../data/train/bfamcv01_segment170_non-neutr...,0,32432,non-neutral-male
3,neutral--bfamcv01--segment173,2.524,../../data/train/bfamcv01_segment173_neutral.wav,0,40384,neutral
4,neutral--bfamcv01--segment177,3.316,../../data/train/bfamcv01_segment177_neutral.wav,0,53056,neutral
...,...,...,...,...,...,...
620,neutral--bpubmn14--segment87,3.676,../../data/train/bpubmn14_segment87_neutral.wav,0,58816,neutral
621,neutral--bpubmn14--segment89,10.376,../../data/train/bpubmn14_segment89_neutral.wav,0,166016,neutral
622,neutral--bpubmn14--segment92,3.177,../../data/train/bpubmn14_segment92_neutral.wav,0,50832,neutral
623,neutral--bpubmn14--segment95,8.254,../../data/train/bpubmn14_segment95_neutral.wav,0,132064,neutral


In [117]:
from sklearn.model_selection import train_test_split
train, dev, y_train, y_dev = train_test_split(
                        df, df.spk_id, 
                        test_size=0.1, 
                        random_state=42)

In [119]:
train.to_csv('train.csv', index=False)
dev.to_csv('dev.csv', index=False)